ログ用画像保存先指定

In [1]:
import os
import datetime

blocked_dir = 'log/blocked'
free_dir = 'log/free'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')
    
try:
    with open('log/' + datetime.datetime.now().strftime('%Y%m%d')+'_log.txt','x') as f:
        f.write('image_name' +  '	' + 'classification' + '	' +'prob_blocked')
except FileExistsError:
    print('Logfile already exist')    

Directories not created becasue they already exist


#Load the trained model(学習済みモデルの読み込み)

We'll assumed that you've already downloaded the best_model.pth to your workstation as instructed in the training notebook. Now, you should upload this model into this notebook's directory by using the Jupyter Lab upload tool. Once that's finished there should be a file named best_model.pth in this notebook's directory. 

(追加) ローカルの学習なので、すでにbest_model.pthがローカルに存在している事を前提とします。


Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model. This should look very familiar from the training notebook.

(訳) Pytorch modelの初期化を下記コードでおこないます。


In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

次に、best_model.pthから、学習した重みを読み込みます

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

現在、モデルの重みは、CPUメモリーで実行されているので、下記コードでGPUに転送します。

In [ ]:
device=torch.device('cuda')
model=model.to(device)

Create the preprocessing function(事前処理関数の作成)

We have now loaded our model, but there's a slight issue. The format that we trained our model doesnt exactly match the format of the camera. To do that, we need to do some preprocessing. This involves the following steps

(訳) モデルを読み込みましたが、まだわずかな問題があります。学習済みモデルのフォーマットと、カメラの形式と完全に一致しません。これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。
1.Convert from BGR to RGB
2.Convert from HWC layout to CHW layout
3.Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4.Transfer the data from CPU memory to GPU memory
5.Add a batch dimension

(訳)
1.BGRからRGBに変換
2.HWC layoutからCHW layoutに変換
3.トレーニング中に使ったのと同じパラメーターを使用して正規化します（カメラは[0、255]の範囲の値を提供し、ロードされた画像は[0、1]の範囲でトレーニングするため、255.0でスケーリングする必要があります
4.dataをCPUメモリからGPUメモリに転送します
5.バッチディメンションを追加する

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

カメラ起動

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

操作ボタン定義

In [3]:
import ipywidgets.widgets as widgets

button_layout = widgets.Layout(width='96px', height='48px')

capture_button = widgets.Button(description='Capture', button_style='info', layout=button_layout)
forward_button = widgets.Button(description='Forward', button_style='success', layout=button_layout)
stop_button = widgets.Button(description='Stop', button_style='danger', layout=button_layout)
left_button = widgets.Button(description='Left', button_style='success', layout=button_layout)
right_button = widgets.Button(description='Right', button_style='success', layout=button_layout)
back_button = widgets.Button(description='Back', button_style='success', layout=button_layout)

blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0,step=0.01, orientation='vertical')
speed_slider=widgets.FloatSlider(description='speed',value=0.3,min=0,max=1,step=0.01)
turn_slider=widgets.FloatSlider(description='turn',value=0.3,min=0,max=1,step=0.01)

画像保存メソッドの定義

In [5]:
import datetime
im_name='hoge'

def save_snapshot(directory):
    global im_name
    cap_image=image
    im_name=datetime.datetime.now().strftime('%Y%m%d_%H%M%S_%f') + '.jpg'
    image_path = os.path.join(directory, im_name)
    with open(image_path, 'wb') as f:
        f.write(cap_image.value)

def save_log(classification):
    global im_name,blocked_slider
    write_log=open(os.path.join('log', datetime.datetime.now().strftime('%Y%m%d')+'_log.txt'),'a')
    write_log.write('\n'+im_name +  '	' + classification+  '	' + str(blocked_slider.value))
    write_log.close()
    
def capture_snapshot():
    global free_dir,blocked_dir,blocked_slider
    if blocked_slider.value < 0.5:
        save_snapshot(free_dir)
        save_log('free')
    else:
        save_snapshot(blocked_dir)
        save_log('blocked')
        
#Captureボタンに動作追加
capture_button.on_click(lambda x: capture_snapshot())

Robotインスタンス生成

In [ ]:
from jetbot import Robot
robot = Robot()

各ボタン動作定義

In [ ]:
import torch.nn.functional as F
import time

def set_forward(change):
    global robot,speed_slider
    robot.stop()
    robot.forward(speed_slider.value)

def set_back(change):
    global robot,speed_slider
    robot.stop()
    robot.backward(speed_slider.value)
    
def set_left(change):
    global robot,turn_slider
    robot.stop()
    robot.left(turn_slider.value)
    time.sleep(0.005)
    robot.stop()

def set_right(change):
    global robot,turn_slider
    robot.stop()
    robot.right(turn_slider.value)
    time.sleep(0.005)
    robot.stop()

def set_stop(change):
    global robot
    robot.stop()
    
def update(change):
    global blocked_slider
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)

    prob_blocked = float(y.flatten()[0])
    blocked_slider.value = prob_blocked
    time.sleep(0.001)
    
update({'new': camera.value})  # we call the function once to intialize

forward_button.on_click(set_forward)
back_button.on_click(set_back)
left_button.on_click(set_left)
right_button.on_click(set_right)
stop_button.on_click(set_stop)

操作パネル表示

In [6]:
display(widgets.HBox([image,blocked_slider]))
control_box_1=widgets.HBox([capture_button,forward_button])
v_control_box=widgets.VBox([stop_button,back_button])
control_box_2=widgets.HBox([left_button,v_control_box,right_button])
display(widgets.VBox([control_box_1,control_box_2]))
display(speed_slider)
display(turn_slider)

FloatSlider(value=0.3, description='speed', max=1.0, step=0.01)

FloatSlider(value=0.3, description='turn', max=1.0, step=0.01)

デモ開始

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

デモ終了

In [ ]:
camera.unobserve(update, names='value')
robot.left_motor.value = 0.0
robot.right_motor.value = 0.0
robot.stop()